(containers)=
# Docker Containers

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/ok-transformer/blob/master/docs/nb/notes/containers.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/ok-transformer?style=social)](https://github.com/particle1331/ok-transformer)

---

**Readings:** [[Docker Guide]](https://docs.docker.com/language/python/)

## Introduction

Containerization solves the problem of running applications consistently with multiple dependencies on the same machine, or across multiple machines, by enabling **reproducible builds** of applications running in **lightweight isolated environments**. Moreover, containers can be easily pulled by other machines from a **container registry**. This is important for development and collaboration. 
Note that this assumes each machine runs a **container runtime**.
In this notebook, we will use [Docker](https://www.docker.com/) which provides an ecosystem for efficiently working with containers.

### Hello world

The following example demonstrates building and running a container:

In [1]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally


latest: Pulling from library/hello-world

fc919002: Pull complete 195kB/3.195kBBDigest: sha256:ac69084025c660510933cca701f615283cdbb3aa0963188770b54c31c8962493
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.do

The above message tells the entire process of how the `hello-world` container eventually is able to run on our machine. The image was pulled on [Docker Hub](https://hub.docker.com/) which is a registry of Docker images. Note that the creation of images occurs locally since the local machine is also our compute layer. 

The container proceeds to run its default command, i.e. execute the `/hello` program that prints the message on the terminal. The `hello-world` image produces a minimal container whose sole purpose is to print this message.

```{figure} containers/img/00-helloworld.svg
---
name: helloworld
width: 600px
---
Anatomy of a Docker image and the resulting `hello-world` container in the context of the Linux kernel. Note the specific partition on the hard disk for the filesystem of the image.  
```

An **image** is essentially a filesystem snapshot with startup commands. This can be thought of as a read-only template which provides the daemon a set of instructions for creating a container. A **container** on the other hand is a running process in the Linux VM with partitioned hardware resources allocated by the kernel.

**Remark.** It would be significantly faster to run the `hello-world` container a second time since Docker uses a **cache** to build it. This makes sense since multiple containers are usually created from the same image.

### Interactive mode

As mentioned, containers have **isolated filesystems** by default. This means we can blow up a container and just create a fresh healthy container from the same image. This also ensures that our running processes will not affect the host computer which can be running other important processes. Running an [ubuntu](https://hub.docker.com/_/ubuntu) container in **detached** (`-d`) and **interactive mode** (`-it`): 

In [2]:
!docker run -d -it --name ubuntu0 ubuntu

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu

Digest: sha256:6042500cf4b44023ea1894effe7890666b0c5c7871ed83a97c36c76ae560bb9b[1A
Status: Downloaded newer image for ubuntu:latest
01aa51ea6dee2d6979609d9f3695ed0a150723283943c64c41a40e86867acaab


This allows us to use the CLI inside the container using `docker exec`:

In [3]:
!docker exec ubuntu0 ls -C
!docker exec ubuntu0 rm -rf bin/ls
!docker exec ubuntu0 ls -C
!docker stop ubuntu0 > /dev/null

bin   dev  home  media	opt   root  sbin  sys  usr
boot  etc  lib	 mnt	proc  run   srv   tmp  var
OCI runtime exec failed: exec failed: unable to start container process: exec: "ls": executable file not found in $PATH: unknown


Creating a fresh container that can run `ls`. Note that the container ID is different:

In [4]:
!docker run -d -it --name ubuntu1 ubuntu
!docker exec ubuntu1 ls -C
!docker stop ubuntu1 > /dev/null

68379450fe0cd1e6ae70bd30b739107f8e0d235bc7a0c0325498cd3ce3e12537
bin   dev  home  media	opt   root  sbin  sys  usr
boot  etc  lib	 mnt	proc  run   srv   tmp  var


### Other commands

Listing all containers and images:

In [5]:
!docker ps --all

CONTAINER ID   IMAGE         COMMAND       CREATED          STATUS                                PORTS     NAMES
68379450fe0c   ubuntu        "/bin/bash"   12 seconds ago   Exited (137) Less than a second ago             ubuntu1
01aa51ea6dee   ubuntu        "/bin/bash"   25 seconds ago   Exited (137) 12 seconds ago                     ubuntu0
587138c99675   hello-world   "/hello"      42 seconds ago   Exited (0) 41 seconds ago                       confident_gagarin


In [6]:
!docker image ls

REPOSITORY    TAG       IMAGE ID       CREATED        SIZE
ubuntu        latest    da935f064913   2 weeks ago    69.3MB
hello-world   latest    ee301c921b8a   7 months ago   9.14kB


**Remark.** The `hello-world` container immediately exited after running with exit status zero since no errors were encountered. The other containers continue running since they are run in interactive mode. 

To stop containers, we can use either `stop` or `kill`. The `stop` command sends a SIGTERM to the running process. This gives 10 seconds for cleanup, then a fallback SIGKILL is sent to immediately terminate the process. See {numref}`lifecycle` and [restart policies](https://docs.docker.com/engine/reference/run/#restart-policies---restart) docs.

```{figure} containers/img/00-lifecycle.png
---
name: lifecycle
---
Complete Docker container lifecycle. [[source](https://docker-saigon.github.io/post/Docker-Internals/)]

```

## Docker build

Throughout the above examples we have been using public images from Docker Hub. 
In this section, we create our own images for running our own containers. Our custom images can be pushed to container
repositories, such as Docker Hub or [ECR](https://aws.amazon.com/ecr/), which our servers can pull 
to run our containers remotely. To do this, Docker requires us to create a `Dockerfile` which specifies the container build process.

In [7]:
import os; os.chdir("./containers/")
!tree ./simple-fastapi -I __pycache__

./simple-fastapi
├── Dockerfile
├── requirements.txt
└── src
    └── main.py

2 directories, 3 files


The web app simply prints a message when the root URI is called:

In [8]:
!pygmentize ./simple-fastapi/src/main.py

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello world!"}


### Dockerfile

The following `Dockerfile` uses `python:3.10-slim` as **base image**. This `slim` image is a smaller version of a container running Python 3.10, but still larger than `alpine`. The next lines serve to modify the base image. First, it specifies `/code` as the **working directory**. This is where all subsequent build commands will be executed. 

The copy command copies files in the build folder to a path relative to the working directory. Next, we call `pip` with certain flags so that it does not cache the installs, making the container smaller. Note that we use `ENTRYPOINT` instead of `CMD`. The latter can be [overridden during run](https://spacelift.io/blog/docker-entrypoint-vs-cmd).

In [9]:
!pygmentize ./simple-fastapi/Dockerfile

FROM python:3.10-slim

WORKDIR /code

COPY ./requirements.txt ./
RUN pip install --no-cache-dir --upgrade -r requirements.txt

COPY ./src ./src

ENTRYPOINT ["uvicorn", "src.main:app", "--host", "0.0.0.0", "--port", "80"]


Building the image. Using the `-t` flag, we add an **image path** (this defaults to the `latest` [tag](https://docs.docker.com/engine/reference/commandline/tag/)):

In [10]:
!docker build ./simple-fastapi -t okt/simple-fastapi

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

### Cached layers

The order of Dockerfile instructions matters. Each instruction in a Dockerfile roughly translates to an **image layer**. The layers are cached in the build process. Hence, changes in one layer destroys the cache of subsequent layers. This is known as **cache busting**. This explains why requirements are installed first in the `Dockerfile` before source code is copied, so that modifying the source does not result in reinstalling the dependencies ({numref}`cache`).

The following created timestamps indicate cached layers in our previous build:

In [11]:
!docker history okt/simple-fastapi

IMAGE          CREATED             CREATED BY                                      SIZE      COMMENT
67acde1baf94   About an hour ago   ENTRYPOINT ["uvicorn" "src.main:app" "--host…   0B        buildkit.dockerfile.v0
<missing>      About an hour ago   COPY ./src ./src # buildkit                     425B      buildkit.dockerfile.v0
<missing>      4 hours ago         RUN /bin/sh -c pip install --no-cache-dir --…   18.7MB    buildkit.dockerfile.v0
<missing>      4 hours ago         COPY ./requirements.txt ./ # buildkit           25B       buildkit.dockerfile.v0
<missing>      4 hours ago         WORKDIR /code                                   0B        buildkit.dockerfile.v0
<missing>      2 months ago        CMD ["python3"]                                 0B        buildkit.dockerfile.v0
<missing>      2 months ago        RUN /bin/sh -c set -eux;   savedAptMark="$(a…   12.2MB    buildkit.dockerfile.v0
<missing>      2 months ago        ENV PYTHON_GET_PIP_SHA256=9cc01665956d22b3bf…   0B  

<br>

```{figure} containers/img/layers.png
---
name: layers
width: 600px
---
Dockerfile translates into a stack of layers in a container image. [Source](https://docs.docker.com/build/guide/layers/)
```

```{figure} containers/img/00-cache-busting.svg
---
name: cache
width: 600px
---
Busting an expensive cached layer (left). Cache optimized version (right).
```

Listing the built image:

In [12]:
!docker image ls

REPOSITORY           TAG       IMAGE ID       CREATED             SIZE
okt/simple-fastapi   latest    67acde1baf94   About an hour ago   173MB
ubuntu               latest    da935f064913   2 weeks ago         69.3MB
hello-world          latest    ee301c921b8a   7 months ago        9.14kB


### Port mapping

Note that the app runs in port `0.0.0.0:80` inside the container. We will expose this to our local machine by **port mapping** it to `localhost:8000`. Running the image in detached mode:

In [13]:
!docker run -d -p 8000:80 --name fastapi okt/simple-fastapi:latest

15692290b3f217c0adee9ad011c3e268f3ab769430413e0c2bca60ad4740b883


In [14]:
!docker ps

CONTAINER ID   IMAGE                       COMMAND                  CREATED         STATUS                  PORTS                  NAMES
15692290b3f2   okt/simple-fastapi:latest   "uvicorn src.main:ap…"   2 seconds ago   Up Less than a second   0.0.0.0:8000->80/tcp   fastapi


In [15]:
import time
time.sleep(2)

Trying it out:

In [16]:
!http :8000

HTTP/1.1 200 OK
content-length: 26
content-type: application/json
date: Wed, 27 Dec 2023 23:26:06 GMT
server: uvicorn

{
    "message": "Hello world!"
}




## Dev environment

This section deals with quality of life improvement for developing with Docker. For example, any local changes to `main.py` will not affect the correspondding script in the container. This can make development difficult with time-consuming rebuilds, e.g. with large images or large files. Finally, our local IDE will not have features like autocomplete and will generally compain of missing packages.

```{figure} ./containers/img/vs-code-module-not-found.png
---
name: vs-code-module-not-found
width: 1200px
---
Module not found. One solution is to install the requirements in a local virtual env. But this is not ideal if you want straightforward reproducibility.
```

### Volumes

The issue with code changes and data is fixed by using **volumes**. This will allow changes in the local filesystem to be reflected within the container (since files are mirrored between the two directories). In our case, the `--reload` flag is essential to avoid manually restarting the uvicorn server inside the container.

In [17]:
!tree $(pwd)/simple-fastapi -I __pycache__

/Users/particle1331/code/ok-transformer/docs/nb/notes/containers/simple-fastapi
├── Dockerfile
├── requirements.txt
└── src
    └── main.py

2 directories, 3 files


Running `docker run` with `-v` flag for mapping volumes and a command argument `--reload` which is appended to the entrypoint. Note the `--reload` flag is not in the `Dockerfile` since this is not suitable for prod.

In [18]:
!docker rm -f fastapi >> /dev/null  # delete prev container
!docker run -d -p 8000:80 -v $(pwd)/simple-fastapi:/code --name fastapi okt/simple-fastapi:latest --reload

ec883f3a77f9e58bbc0de5ad03dbc1f2d00be10710fc31caf68bfbd948be333c


Modifying the main file:

In [19]:
%%writefile ./simple-fastapi/src/main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello world + 123!"}    # (!)

Overwriting ./simple-fastapi/src/main.py


In [20]:
import time
time.sleep(5)

Logs show that the application is reloading:

In [21]:
!docker logs fastapi

INFO:     Will watch for changes in these directories: ['/code']
INFO:     Uvicorn running on http://0.0.0.0:80 (Press CTRL+C to quit)
INFO:     Started reloader process [1] using StatReload
INFO:     Started server process [8]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


The response should change without rebuild (the ff. uses [httpie](https://httpie.io/docs/cli/universal)):

In [22]:
!http :8000

HTTP/1.1 200 OK
content-length: 32
content-type: application/json
date: Wed, 27 Dec 2023 23:26:16 GMT
server: uvicorn

{
    "message": "Hello world + 123!"
}




In [23]:
%%writefile ./simple-fastapi/src/main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello world!"}

Overwriting ./simple-fastapi/src/main.py


### Remote container IDE

To follow this section, you have to install [Docker](https://marketplace.visualstudio.com/items?itemName=ms-azuretools.vscode-docker) and [Dev Containers](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-containers) extensions in VS Code. To use an IDE with a running container, you can click on the lower left button or press CTRL+SHIFT+P and type "Dev Containers: Attach to running container". This opens up a new window. You have to install the [Python extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python) once to get IDE features. The experience is the same as when you SSH into a remote server. 

```{figure} ./containers/img/vs-code-completion.png
---
name: vs-code-completion
width: 1200px
---
IDE attached to the container with code completion and other useful features.
If the container is mapped to a volume, then any change made using the IDE is mirrored in the
host directory.
```

In [24]:
!docker rm -f fastapi >> /dev/null

### Debugging

**Note:** *Refer to the following section on docker compose for the application used here.*

For debugging, we use https://github.com/microsoft/debugpy. Run the application defined in the following compose file. This overrides the Dockerfile entrypoint and simply runs the uvicorn server with debugpy client listening on port 5678.

```yaml
# containers/compose/docker-compose.debug.yml
version: "3"
services:
  fastapi-server:
    build: app
    restart: on-failure
    entrypoint: ""
    command: ["sh", "-c", "pip install debugpy -t /tmp && python /tmp/debugpy --wait-for-client --listen 0.0.0.0:5678 -m uvicorn src.main:app --host 0.0.0.0 --port 8000 --reload"]
    ports:
      - 8080:8000
      - 5678:5678
    volumes:
      - ./app:/code
    depends_on:
      - redis

  redis:
    image: redis:alpine
```

To run this:

```
docker compose -f docker-compose.debug.yml up
```

Remote attach to the FastAPI container using VS Code. In the 🐞 tab of the remote IDE, click "create a launch.json file". Select "Remote Attach" and enter "localhost" with port "5678". Start the debugger and add breakpoints. Then, we can make the relevant request to debug it ({numref}`vs-code-debugger`).

```{figure} ./containers/img/vs-code-debugger.png
---
name: vs-code-debugger
width: 1200px
---
Debugger running. Breakpoints are triggered after a GET request to `localhost:8080`.
```

## Docker compose

Multi-container applications require configuring setup and tear down of run and builds, volumes, as well as networking between multiple services. This can be tedious to do using Docker CLI commands especially during development. [Docker Compose](https://docs.docker.com/compose/) allows us to collect all configurations in a YAML file. This takes care of networking between containers as well as logging and status monitors for the whole ensemble.

### Files

The web server is in its own directory containing its corresponding Dockerfile:

In [25]:
!tree ./compose -I __pycache__

./compose
├── app
│   ├── Dockerfile
│   ├── requirements.txt
│   └── src
│       └── main.py
└── docker-compose.yml

3 directories, 4 files


This simply tracks the visit count along with a message. For storing the counts, we will use a [redis](https://redis.io/docs/connect/clients/python/) database. Note that since compose takes care of networking, it suffices to use the container name (see compose file below) as host for the redis client:

In [26]:
!pygmentize ./compose/app/src/main.py

import redis
from fastapi import FastAPI

app = FastAPI()
r = redis.Redis(host="redis", port=6379)

if not r.exists("visits"):
    r.set("visits", 0)


@app.get("/")
def root():
    visits = int(r.get("visits")) + 1
    r.set("visits", visits)
    return {
        "message": "Hello world!",
        "visit_count": r.get("visits"),
    }


Note that paths in Dockerfile and compose files are relative:

In [27]:
!pygmentize ./compose/app/Dockerfile

FROM python:3.10-slim

WORKDIR /code

COPY ./requirements.txt ./

RUN pip install --no-cache-dir --upgrade -r requirements.txt

COPY ./src ./src

ENTRYPOINT ["uvicorn", "src.main:app"]


The compose file simply lists the services and its run configurations. Notice that port and volume mapping are already specified here,
as well as run commands. Moreover, the dependence of the web server to the database is stated. This means the `redis` service is started first.

In [28]:
!pygmentize ./compose/docker-compose.yml

version: "3"
services:
  fastapi-server:
    build: app
    restart: on-failure
    command: "--host 0.0.0.0 --port 80 --reload"
    ports:
      - 8080:80
      - 5678:5678
    volumes:
      - ./app:/code
    depends_on:
      - redis

  redis:
    image: redis:alpine


**Remark.** Docker compose is [typically used](https://docs.docker.com/compose/features-uses/#common-use-cases-of-docker-compose) for development and automated testing use cases. So it's okay to have `--reload` hard coded here.

### Compose up

Starting the multi-container application. The `--build` flag is optional and is used to rebuild containers from images. Again, we use `-d` to run it in detached mode. Here, we also change the build context. Otherwise, we would need to add `-f PATH` to point to the path of the compose file each time we use `docker compose`.

In [29]:
os.chdir("./compose")
!docker-compose up -d --build

[+] Running 0/0
 ⠙ redis Pulling                                                           0.1s
[+] Running 0/1
 ⠹ redis Pulling                                                           0.2s
[+] Running 0/1
 ⠸ redis Pulling                                                           0.3s
[+] Running 0/1
 ⠼ redis Pulling                                                           0.4s
[+] Running 0/1
 ⠴ redis Pulling                                                           0.5s
[+] Running 0/1
 ⠦ redis Pulling                                                           0.6s
[+] Running 0/1
 ⠧ redis Pulling                                                           0.7s
[+] Running 0/1
 ⠇ redis Pulling                                                           0.8s
[+] Running 0/1
 ⠏ redis Pulling                                                           0.9s
[+] Running 0/1
 ⠋ redis Pulling                                                           1.0s
[+] Running 0/1
 ⠙ redis Pulling        

**Remark.** Services which are still up will not be rebuilt. As such, these services can persist data and state while rebuilt services have reset state.

In [30]:
!docker compose ps

NAME                       IMAGE                    COMMAND                  SERVICE             CREATED             STATUS                  PORTS
compose-fastapi-server-1   compose-fastapi-server   "uvicorn src.main:ap…"   fastapi-server      1 second ago        Up Less than a second   0.0.0.0:5678->5678/tcp, 0.0.0.0:8080->80/tcp
compose-redis-1            redis:alpine             "docker-entrypoint.s…"   redis               2 seconds ago       Up Less than a second   6379/tcp


In [31]:
time.sleep(1)

Making multiple requests:

In [32]:
!http :8080

HTTP/1.1 200 OK
content-length: 44
content-type: application/json
date: Wed, 27 Dec 2023 23:26:41 GMT
server: uvicorn

{
    "message": "Hello world!",
    "visit_count": "1"
}




In [33]:
!http :8080

HTTP/1.1 200 OK
content-length: 44
content-type: application/json
date: Wed, 27 Dec 2023 23:26:41 GMT
server: uvicorn

{
    "message": "Hello world!",
    "visit_count": "2"
}




Monitoring running services resource usage:

In [34]:
!docker stats --no-stream $(docker ps --format "{{.Names}}" | grep -w 'compose')

CONTAINER ID   NAME                       CPU %     MEM USAGE / LIMIT     MEM %     NET I/O           BLOCK I/O   PIDS
8176f2fae142   compose-fastapi-server-1   1.19%     54.91MiB / 3.841GiB   1.40%     3.5kB / 2.89kB    0B / 0B     4
5f79ae484a67   compose-redis-1            0.28%     2.82MiB / 3.841GiB    0.07%     2.92kB / 1.36kB   0B / 0B     5


Teardown:

In [35]:
!docker compose down

[+] Running 0/0
 ⠋ Container compose-fastapi-server-1  Stopping                            0.1s
[+] Running 0/1
 ⠙ Container compose-fastapi-server-1  Stopping                            0.2s
[+] Running 0/1
 ⠹ Container compose-fastapi-server-1  Stopping                            0.3s
[+] Running 0/1
 ⠸ Container compose-fastapi-server-1  Stopping                            0.4s
[+] Running 0/1
 ⠼ Container compose-fastapi-server-1  Stopping                            0.5s
[+] Running 0/1
 ⠴ Container compose-fastapi-server-1  Stopping                            0.6s
[+] Running 1/1
 ⠿ Container compose-fastapi-server-1  Removed                             0.6s
 ⠋ Container compose-redis-1           Stop...                             0.0s
[+] Running 1/2
 ⠿ Container compose-fastapi-server-1  Removed                             0.6s
 ⠙ Container compose-redis-1           Stop...                             0.1s
[+] Running 1/2
 ⠿ Container compose-fastapi-server-1  Removed          

In [36]:
!docker rm ubuntu0
!docker rm ubuntu1

ubuntu0
ubuntu1


## Readings: Best practices

* [What is the best way to pass AWS credentials to a Docker container?](https://stackoverflow.com/a/76191745/1091950)
* [Security best practices](https://docs.docker.com/develop/security-best-practices/)
* [Optimizing builds with cache management](https://docs.docker.com/build/cache/)
* [Docker Best Practices for Python Developers](https://testdriven.io/blog/docker-best-practices/)